In [1]:
from os import listdir
from os.path import isfile, join

mypath = "/Volumes/TOSHIBA/Rekordbox_database"

files = [f for f in listdir(mypath) if not (f).startswith(".")]

In [2]:
import requests
import discogs_client
d = discogs_client.Client('tracklist_explorer', user_token='TQnLrfoPbUSIPjrwWqIkZSkuJczEwsHtQXbdegwL')

In [3]:
import pandas as pd

df = pd.DataFrame(files, columns=["title"])

In [4]:
df

,title
0,SPIRINET - ENERGY FORMULA -KHT007
1,Babe Roots feat Kojo Neatness - Be Still - Raw...
2,CASH FROM HASH - CASH FROM HASH - AXON
3,Jay Glass Dubs - Epitaph
4,"Mina, Bryte, King Doudou - Mina ft. Bryte - Se..."
...,...
648,Squito Babe - Palombaro [Explicit]
649,RSD-Corner_Dub_(Blue_and_Red_Mix)_&_Pretty_Bri...
650,Minor Science - Whities 008 [Ep] (2016)
651,(1997) Dj Jubilee - Eternal Dark (Unknown Grou...


In [5]:
import re

In [6]:
df["title"] = [x[:-4] if x.endswith(".mp3") else x for x in df["title"]]
df["title"] = [x.replace("feat", "") for x in df["title"]]
df["title"] = [x.replace("'", "") for x in df["title"]]
df["title"] = [x.replace("\"", "") for x in df["title"]]
df["title"] = [x.replace("-", " ") for x in df["title"]]
df["title"] = [x.replace("_", " ") for x in df["title"]]
df["title"] = [x.replace("&", " ") for x in df["title"]]
df["title"] = [re.sub("[\(\[].*?[\)\]]", "", x) for x in df["title"]]

In [7]:
df

,title
0,SPIRINET ENERGY FORMULA KHT007
1,Babe Roots Kojo Neatness Be Still Rawness
2,CASH FROM HASH CASH FROM HASH AXON
3,Jay Glass Dubs Epitaph
4,"Mina, Bryte, King Doudou Mina ft. Bryte Se..."
...,...
648,Squito Babe Palombaro
649,RSD Corner Dub Pretty Bright Light
650,Minor Science Whities 008
651,Dj Jubilee Eternal Dark


In [8]:
df.to_csv("df_clean_titles.csv")

works with single tracks as well

In [9]:
release = d.search('Clara! y Maoupa - Truenos', type='release')[0]

In [10]:
release.styles

['Experimental', 'Reggaeton']

In [11]:
styles = []
genres = [] 
for idx,i in df.iterrows():
    try:
        release = d.search(i["title"], type='release')[0]
        styles.append(release.styles)
        genres.append(release.genres)
    except:
        styles.append("error")
        genres.append("error")

In [12]:
df["style"] = styles
df["genres"] = genres

In [13]:
df.to_csv("df.csv")

In [14]:
df["style"].value_counts()

error                                             119
[Techno]                                           34
[Dubstep]                                          34
[House]                                            11
[Experimental]                                      7
                                                 ... 
[Electro, Techno, Industrial, Acid, Breakbeat]      1
[Bass Music, House]                                 1
[Grime, Dub Techno, Breaks, Leftfield]              1
[Downtempo, EBM, Dub]                               1
[Experimental, Glitch]                              1
Name: style, Length: 352, dtype: int64

In [15]:
errors = df[df["style"] == "error"]

In [16]:
len(errors)

119

In [17]:
errors.to_csv("errors.csv")

In [18]:
df.loc[errors.index]

,title,style,genres
2,CASH FROM HASH CASH FROM HASH AXON,error,error
4,"Mina, Bryte, King Doudou Mina ft. Bryte Se...",error,error
7,scopeotaku 4040 d,error,error
11,King Doudou Toa,error,error
18,La Zowi Élite,error,error
...,...,...,...
637,Notte Brigante NB03t Simplists Over SImp...,error,error
639,ArtSaves Home Is Where the Hatred Is,error,error
641,Abu AMA INTERNATiONAL ROKKAZ,error,error
648,Squito Babe Palombaro,error,error


In [22]:
df_final = df.drop(index=errors.index)
df_final

,title,style,genres
0,SPIRINET ENERGY FORMULA KHT007,[House],[Pop]
1,Babe Roots Kojo Neatness Be Still Rawness,"[Dub, Dub Techno, Dubstep]","[Electronic, Reggae]"
3,Jay Glass Dubs Epitaph,"[Dub, Dancehall, Experimental, Industrial, Tri...","[Electronic, Reggae]"
5,Pearson Sound X Clara! Mi Cuerpo,"[Reggaeton, House, Dubstep]","[Electronic, Latin]"
6,DJ Seinfeld Time Spent Away From U,[House],[Electronic]
...,...,...,...
646,Gesloten Cirkel Gesloten Cirkel,"[House, Electro, Acid, Techno]",[Electronic]
647,James Blake If The Car Beside You Moves Ahead,"[Experimental, Glitch]",[Electronic]
649,RSD Corner Dub Pretty Bright Light,[Dubstep],[Electronic]
650,Minor Science Whities 008,[Techno],[Electronic]


In [33]:
list(df_final["style"])

[['House'],
 ['Dub', 'Dub Techno', 'Dubstep'],
 ['Dub', 'Dancehall', 'Experimental', 'Industrial', 'Trip Hop', 'Leftfield'],
 ['Reggaeton', 'House', 'Dubstep'],
 ['House'],
 ['Downtempo', 'Tribal'],
 ['Dubstep', 'Dub'],
 ['Trap', 'Industrial', 'Post-Punk'],
 ['Soul', 'Rhythm & Blues', 'Garage Rock'],
 ['Breakbeat', 'Tribal', 'Techno'],
 ['Acid', 'IDM', 'Techno'],
 ['Dubstep'],
 ['Grime', 'UK Garage'],
 ['Dubstep'],
 ['Techno', 'EBM'],
 ['Abstract'],
 ['Electro', 'Industrial', 'Leftfield'],
 ['Dubstep'],
 ['Pop Rock'],
 ['Dub', 'Ambient', 'Leftfield', 'Techno'],
 ['Trip Hop'],
 ['Abstract'],
 ['Speech', 'Religious'],
 ['Grime', 'Techno', 'Dubstep'],
 ['Disco', 'House'],
 ['Gospel', 'Industrial', 'Noise'],
 ['Drum n Bass', 'Techno', 'Halftime'],
 ['Dubstep', 'Grime', 'Ragga', 'Dancehall'],
 ['Dancehall', 'Industrial', 'Breakbeat'],
 ['House'],
 ['Techno'],
 ['Tribal', 'Techno'],
 ['Techno', 'Breaks', 'Hardcore'],
 ['Dubstep'],
 ['Techno', 'Dubstep'],
 ['House', 'Progressive House', 'Bass

In [38]:
df_exploded = df_final.explode("style").drop("genres", axis=1)

In [40]:
set(df_exploded["style"])

{'Abstract',
 'Acid',
 'Acid House',
 'African',
 'Afro-Cuban Jazz',
 'Alternative Rock',
 'Ambient',
 'Art Rock',
 'Avant-garde Jazz',
 'Avantgarde',
 'Balearic',
 'Ballad',
 'Bass Music',
 'Bassline',
 'Berlin-School',
 'Big Beat',
 'Bleep',
 'Blues Rock',
 'Bop',
 'Breakbeat',
 'Breakcore',
 'Breaks',
 'Broken Beat',
 'Bubbling',
 'Chillwave',
 'Classic Rock',
 'Classical',
 'Coldwave',
 'Conscious',
 'Country Rock',
 'Dancehall',
 'Dark Ambient',
 'Darkwave',
 'Deconstructed Club',
 'Deep House',
 'Deep Techno',
 'Disco',
 'Downtempo',
 'Drill',
 'Drone',
 'Drum n Bass',
 'Dub',
 'Dub Techno',
 'Dubstep',
 'EBM',
 'Electro',
 'Electroacoustic',
 'Ethereal',
 'Euro House',
 'Europop',
 'Experimental',
 'Field Recording',
 'Folk Rock',
 'Footwork',
 'Funk',
 'Future Jazz',
 'Gamelan',
 'Gangsta',
 'Garage House',
 'Garage Rock',
 'Ghetto',
 'Ghetto House',
 'Glam',
 'Glitch',
 'Gospel',
 'Gqom',
 'Grime',
 'Grindcore',
 'Halftime',
 'Hard House',
 'Hard Techno',
 'Hard Trance',
 'Har

In [41]:
df_exploded[df_exploded["style"] == "Ambient"]

,title,style
24,OL Dismeteo,Ambient
50,Pendant Make Me Know You Sweet,Ambient
52,Ancient Plastix S T,Ambient
63,Group Zero Everyone’s Already Come Apart,Ambient
71,Alien + B12 Remix Unreleased 96 00,Ambient
...,...,...
581,Atoloi Papier dArmenie,Ambient
608,Anthony Naples Fog FM,Ambient
624,XIII Permanent Rain,Ambient
629,memotone How Was Your Life,Ambient


In [42]:
mypath = "/Volumes/TOSHIBA/Rekordbox_database"

files = [f for f in listdir(mypath) if not (f).startswith(".")]

In [43]:
df = pd.DataFrame(files, columns=["path"])

In [44]:
df

,path
0,SPIRINET - ENERGY FORMULA -KHT007
1,Babe Roots feat Kojo Neatness - Be Still - Raw...
2,CASH FROM HASH - CASH FROM HASH - AXON
3,Jay Glass Dubs - Epitaph
4,"Mina, Bryte, King Doudou - Mina ft. Bryte - Se..."
...,...
648,Squito Babe - Palombaro [Explicit]
649,RSD-Corner_Dub_(Blue_and_Red_Mix)_&_Pretty_Bri...
650,Minor Science - Whities 008 [Ep] (2016)
651,(1997) Dj Jubilee - Eternal Dark (Unknown Grou...


In [45]:
df_exploded["path"] = df.loc[df_exploded.index]

In [46]:
df_exploded

,title,style,path
0,SPIRINET ENERGY FORMULA KHT007,House,SPIRINET - ENERGY FORMULA -KHT007
1,Babe Roots Kojo Neatness Be Still Rawness,Dub,Babe Roots feat Kojo Neatness - Be Still - Raw...
1,Babe Roots Kojo Neatness Be Still Rawness,Dub Techno,Babe Roots feat Kojo Neatness - Be Still - Raw...
1,Babe Roots Kojo Neatness Be Still Rawness,Dubstep,Babe Roots feat Kojo Neatness - Be Still - Raw...
3,Jay Glass Dubs Epitaph,Dub,Jay Glass Dubs - Epitaph
...,...,...,...
647,James Blake If The Car Beside You Moves Ahead,Experimental,James Blake - If The Car Beside You Moves Ahead
647,James Blake If The Car Beside You Moves Ahead,Glitch,James Blake - If The Car Beside You Moves Ahead
649,RSD Corner Dub Pretty Bright Light,Dubstep,RSD-Corner_Dub_(Blue_and_Red_Mix)_&_Pretty_Bri...
650,Minor Science Whities 008,Techno,Minor Science - Whities 008 [Ep] (2016)


In [49]:
from mutagen.mp3 import MP3  
from mutagen.easyid3 import EasyID3  
import mutagen.id3  
from mutagen.id3 import ID3, TIT2, TIT3, TALB, TPE1, TRCK, TYER  
  
import glob  
  
import numpy as np  

In [52]:
import os
example_path = os.path.join(mypath, df_exploded.iloc[0]["path"])
example_path

'/Volumes/TOSHIBA/Rekordbox_database/SPIRINET - ENERGY FORMULA -KHT007'

In [54]:
# extract the file names (including folders)  
# for the mp3s in the album  
filez = glob.glob(os.path.join(example_path, '*.mp3'))
 # print the first element of filez:  
filez[0]  

'/Volumes/TOSHIBA/Rekordbox_database/SPIRINET - ENERGY FORMULA -KHT007/SPIRINET - ENERGY FORMULA -KHT007- - 02 Running Perfectly.mp3'

In [67]:
mp3file = MP3(filez[0], ID3=EasyID3)

In [68]:
mp3file

{'album': ['ENERGY FORMULA [KHT007]'], 'title': ['Running Perfectly'], 'artist': ['SPIRINET'], 'albumartist': ['SPIRINET'], 'tracknumber': ['2'], 'date': ['2021']}

In [60]:
mp3file['genre'] = "Pop"

In [65]:
mp3file.save()

In [64]:
mp3file['artist'] = "SPIRINET"

In [66]:
mp3file

{'album': ['ENERGY FORMULA [KHT007]'], 'title': ['Running Perfectly'], 'artist': ['SPIRINET'], 'albumartist': ['SPIRINET'], 'tracknumber': ['2'], 'genre': ['Pop'], 'date': ['2021']}